## Alcholic Anonymous News articles extraction from The Tribune. Data Extraction of following parameters
- Headline
- Description
- Author
- Published Date
- Category
- Publication
- News
- URL
- Keywords
- Summary

### Importing the necessary Libraries

In [1]:
from selenium.webdriver.chrome.options import Options # enables options in web browser
from selenium import webdriver # web-based automation tool for Python
from newspaper import Article # Article scraping & curation
from bs4 import BeautifulSoup # Python library for pulling data out of HTML and XML files
from requests import get # standard for making HTTP requests in Python
import pandas as pd # library written for data manipulation and analysis
import sys, time #  System-specific parameters and functions

### Creating Empty lists for Alcholic Anonymous News Articles parameters data to be extracted

In [2]:
headlines, descriptions, dates, authors, news, keywords, summaries, urls, category, publication = [], [], [], [], [], [], [], [], [], []

### Finding the total no.of.pages by total no.of articles from google search results¶

In [3]:
keyword = 'Alcoholics Anonymous site:www.tribuneindia.com'

url = 'https://www.google.com/search?q=' + '+'.join(keyword.split())

soup = BeautifulSoup(get(url).text, 'lxml')
try:
    # Extracts the digits if it the resulted number without comma ','. eg: About 680 results (0.23 seconds)
    max_pages = round([int(s) for s in soup.select_one('div#resultStats').text.split() if s.isdigit()][0]/10)
    max_pages = max_pages + 1
except:
    # Extracts the digits if it the resulted number without comma ','. eg: About 1,080 results (0.23 seconds)
    max_pages = round(int(''.join(i for i in soup.select_one('div#resultStats').text if i.isdigit()))/10)
    max_pages = max_pages + 1

### Iterates max_pages value through while loop. Scraping the Articles urls

In [4]:
options = Options()
options.headless = True
browser = webdriver.Chrome(options=options)
browser.get(url)

index = 0

while True:
    try:
        index +=1
        page = browser.page_source
        soup = BeautifulSoup(page, 'lxml')
        linky = [soup.select('.r')[i].a['href'] for i in range(len(soup.select('.r')))]
        urls.extend(linky)
        if index == max_pages:
            break
        browser.find_element_by_xpath('//*[@id="pnnext"]/span[2]').click()
        time.sleep(2)
        sys.stdout.write('\r' + str(index) + ' : ' + str(max_pages) + '\r')
        sys.stdout.flush()
    except:
        pass
    
browser.quit()

### To remove duplicates urls entries in the list by executing below line

In [5]:
urls = list(dict.fromkeys(urls))
print("Total Extracted URL's are" + ' : ' + str(len(urls)), type(urls))

Total Extracted URL's are : 174 <class 'list'>


### Iterates urls through for loop. Scraping the Articles with above parameters

In [6]:
%%time
for index, url in enumerate(urls):
    try:
        # Parse the url to NewsPlease 
        soup = BeautifulSoup(get(url).text, 'lxml')
        article = Article(url)
        article.download()
        article.parse()
        article.nlp()
        
        # Extracts the Headlines
        try:
            try:
                headlines.append(article.title.strip())
            except:
                headlines.append(soup.select_one('meta[property="og:title"]')['content'].strip())
        except:
            headlines.append(None)
            
        # Extracts the Descriptions    
        try:
            try:
                descriptions.append(soup.select_one('meta[property="og:description"]')['content'].strip().replace('\n', ' '))
            except:
                descriptions.append(article.meta_description.strip())
        except:
            descriptions.append(None)
            
        # Extracts the Authors
        try:
            try:
                authors.append(soup.select_one('meta[name="author"]')['content'].strip())
            except:
                authors.append(article.authors.strip())
        except:
            authors.append(None)
        
        # Extracts the published dates
        try:
            try:
                dates.append(soup.select_one('meta[property="article:published_time"]')['content'].strip())
            except:
                dates.append(str(article.publish_date))
        except:
            dates.append(None)
            
        # Extracts the news articles
        try:
            try:
                news.append(soup.select_one('.storyText').text.replace('\n', '').strip())
            except:
                news.append(' '.join(article.text.split()).replace("\'\'"," ").replace("\'", "").replace(" / ", ""))
        except:
            news.append(None)
            
        # Extracts the news category
        try:
            category.append(article.meta_data['category'])
        except:
            category.append(None)
            
        # Extracts the news publication
        try:
            publication.append(article.meta_data['og']['site_name'])
        except:
            publication.append(None)

        # Extracts Keywords and Summaries
        try:
            keywords.append(article.keywords)
            summaries.append(article.summary)
        except:
            keywords.append(None)
            summaries.append(None)
                        
    except:
        headlines.append(None)
        descriptions.append(None)
        authors.append(None)
        dates.append(None)
        category.append(None)
        publication.append(None)
        news.append(None)
        keywords.append(None)
        summaries.append(None)

    sys.stdout.write('\r' + str(index) + ' : ' + str(url) + '\r')
    sys.stdout.flush()

Wall time: 18min 50sibuneindia.com/2014/20140408/dun.htmmaces-for-top-court-verdict-on-panamagate-today/394557.htmll.htmlml.html/619408.html


### Checking Array Length of each list to create DataFrame

In [7]:
print(len(headlines), len(descriptions), len(authors), len(dates), len(category), len(publication), len(news), len(keywords), len(summaries), len(urls))

174 174 174 174 174 174 174 174 174 174


### Creating a csv file after checking array length and droping the missing values from the dataset

In [8]:
if len(headlines) == len(descriptions) == len(authors) == len(dates) == len(news) == len(publication) == len(keywords) == len(summaries) == len(urls) == len(category):
    tbl = pd.DataFrame({'Headlines' : headlines,
                        'Descriptions' : descriptions,
                        'Authors' : authors,
                        'Published_Dates' : dates,
                        'Publication' : publication,
                        'Articles' : news,
                        'category' : category,
                        'Keywords' : keywords,
                        'Summaries' : summaries,
                        'Source_URLs' : urls})
    tbl.dropna()
    path = 'C:\\Users\\GM\\Desktop\\!Code!\\CDRI\\Alcoholics Anonymous\\Data Extraction\\#Dataset\\'
    tbl.to_csv(path+'The_Tribune.csv', index=False)
else:
    print('Array lenght does not match!')

tbl.head()

,Headlines,Descriptions,Authors,Published_Dates,Publication,Articles,category,Keywords,Summaries,Source_URLs
0,Hope for alcoholics through AA: Experts,CHANDIGARH: There is a hope for hopeless alcoh...,Tribune News Service,2016-08-09T02:36:00,Tribuneindia News Service,"Chandigarh, August 8There is a hope for hopele...",{},"[wanting, experts, organisations, medical, pub...","Chandigarh, August 8There is a hope for hopele...",https://www.tribuneindia.com/news/community/ho...
1,When she hits the bottle...,"Today, globally India shows the third largest ...",Tribune News Service,2016-07-31T00:58:00,Tribuneindia News Service,"Linda Chhakchhuak\rToday, globally India shows...",{},"[alcohol, alcoholic, aai, hits, paite, cent, w...","Indeed, leading brands are not hesitant to con...",https://www.tribuneindia.com/news/society/when...
2,"The Tribune, Chandigarh, India",,None,2000-08-08 00:00:00,None,HEALTH Alcoholism is a disease like cancer or...,{},"[alcoholic, tribune, father, sharing, india, a...",The common man has to know that alcoholism is ...,https://www.tribuneindia.com/2000/20000808/cth...
3,Want to quit drinking? Walk into AA special me...,"PANCHKULA: The Alcoholics Anonymous (AA), a wo...",Tribune News Service,2016-05-22T02:29:00,Tribuneindia News Service,"Tribune News ServicePanchkula, May 21The Alcoh...",{},"[fellowship, drinking, walk, group, special, s...","The AA members would share their experiences, ...",https://www.tribuneindia.com/news/community/wa...
4,"Kicking the bottle, one day at a time",The picturesque resort along the gushing water...,Pradeep Sharma,2015-06-21T01:57:00,Tribuneindia News Service,The picturesque resort along the gushing water...,{},"[alcohol, drinking, stop, kicking, help, life,...","However, the disease can be arrested “one day ...",https://www.tribuneindia.com/news/perspective/...


In [9]:
tbl.shape

(174, 10)